In [ ]:
%matplotlib inline
from netCDF4 import Dataset as ncread
import netCDF4 as nc
import numpy as np
from scipy.io import loadmat
import pandas as pd
import h5py
import math
import pandas as pd
from datetime import datetime
from itertools import product
from cftime import DatetimeNoLeap
import xarray as xr

In [ ]:
from preprocess import get_principle_components_and_EOFs
import os
import visualization

In [ ]:
%load_ext autoreload
%autoreload

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams #For changing text properties
import cmocean #A package with beautiful colormaps
import matplotlib.path as mpath

In [ ]:
from datetime import datetime 
from datetime import timedelta
from datetime import date
import time

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.utils import shuffle

In [ ]:
import tensorflow as tf    
#tf.compat.v1.disable_v2_behavior() # <-- HERE !

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Activation, Reshape, Flatten, LSTM, Dense, Dropout, Embedding, Bidirectional, GRU
from tensorflow.keras import Sequential
from tensorflow.keras import initializers, regularizers
from tensorflow.keras import optimizers
from tensorflow.keras import constraints
from tensorflow.keras.layers import Layer, InputSpec

## Data set

In [ ]:
root_data = '/data/volume_2/observational/raw/'

import pathlib
root_results = str(pathlib.Path.home() / 'Results')

#### target 
see https://github.com/AI4S2S/Lorentz_s2spy_workshop/blob/main/preprocess_target.ipynb

In [ ]:
SYY = 1981   # start year, could be changed
EYY = 2021   # end year, could be changed


drop_OND_years = [2005,2007,2018,2004,2006]
take_OND_years = list(np.arange(SYY,EYY+1))
take_OND_years = [y for y in take_OND_years if y not in drop_OND_years]
# drop_MAM_years = [2009,2001,2002,2005,2020]

In [ ]:
# left aligned
t = xr.open_dataset("/data/volume_2/observational/chrips_1981-2021_target_new_left.nc").sel(time = slice(str(SYY),str(EYY)))
# take only OND: oct 1st - Dec 3rd
t = xr.concat([t.sel(time=slice(f"{yyyy}-10-01",f"{yyyy}-12-03")) for yyyy in t.time.dt.year.to_index().unique()],"time")

tp_index = t.binary # 1 drought, 0 no drought: reversed to initial commits

tp_target = t.tp_28d_rm # 28 day rolling mean rain

#### Prepare predictors
- 28-days time series with the last day two weeks before the target day
- dimensionality reduction: EOFs
- each variable has their own region for EOF extraction

In [ ]:
# Predictor data preprocessing
# can select the values and region you want by changing the parameters

file_vars = ['ERA5_t2m', 'era5_t_850hpa', 'era5_z_200hpa', 'era5_z_500hpa', 'sst', 'era5_olr']
#file_vars = ['sst']
header_vars = ['t2m', 't', 'z', 'z', 'sst', 'olr-mean']
#header_vars = ['sst']

# select regions for the individual predictor
lon_slices = [[-16,54],[-30,90],[-30,90],[-30,90],[-180,180],[40,180]]
lat_slices = [[16,0],[30,-20],[-20,30],[-20,30],[40,-20],[-20,20]]

nmode = 5 # for eofs

for file_var, header_var, lon_slice, lat_slice in zip (file_vars, header_vars, lon_slices, lat_slices):
    # use existing
    path = root_results+'/PC_series_n_'+str(nmode)+'_var_'+file_var+'_.nc'
    path_eof = root_results+'/EOF_maps_n_'+str(nmode)+'_var_'+file_var+'_.nc'
    
    if os.path.exists(path):
        continue
    if file_var == 'era5_olr':
        file = xr.open_dataset(root_data+file_var+'_1950_2021_daily_1deg_tropics.nc')
        print('olr')
    else:
        file = xr.open_dataset(root_data+file_var+'_1959-2021_1_12_daily_2.0deg.nc')
        print(header_var)

    if "longitude" in file.coords:
        file = file.rename({"longitude": "lon","latitude": "lat"})

    assert "lat" in file.coords
    assert "lon" in file.coords
    
    # select region
    var_dim = file.sel(lon=slice(lon_slice[0],lon_slice[1]),lat=slice(lat_slice[0],lat_slice[1]))
    
    # todo: train_valid_test_split: exclude test    
    # take years 1980 - 2021 daily and only and 7 day rolling mean
    var_series = var_dim.sel(time=var_dim.time.dt.year.isin([np.arange(SYY,EYY+1)])).rolling(time=7, center=False).mean(skipna=True)
    var_series = var_series.sel(time=var_series.time.dt.year.isin(take_OND_years))

    # remove climatology
    var_anom_series = var_series.groupby("time.dayofyear") - var_series.groupby("time.dayofyear").mean("time",skipna=True)
    
    # use the months you want (base on how long the time series used as predictors)
    var_anom_sel = var_anom_series.sel(time=var_anom_series.time.dt.month.isin([7,8,9,10,11]))[header_var]
    
    # Apply EOF
    if file_var == "era5_z_500hpa":
        header_var = "z500"
    pc_xr, EOF = get_principle_components_and_EOFs(var_anom_sel, nmode=nmode)
    
    pc_xr = pc_xr.assign_coords(mode=[str(header_var)+'_'+str(int(m)) for m in pc_xr.mode])
    EOF = EOF.assign_coords(mode=[str(header_var)+'_'+str(int(m)) for m in EOF.mode])
    # save to disk
    pc_xr.to_netcdf(path)
    EOF.to_netcdf(path_eof)

In [ ]:
# quick viz
# pc_xr.plot(hue="mode", figsize=(20,3))
# var_anom_series[header_var].isel(time=122).plot(size=5, aspect=4)

In [ ]:
#exec(open('/home/mpyrina/Notebooks/Lorentz_workshop/L_functions.py').read())

from L_functions import sel_train_data_lead

#Create predictor multi-file
nc_in_file='PC_serie*.nc'
dim_to_stack='mode'
pc_xr = xr.open_mfdataset(root_results+"/"+nc_in_file,concat_dim=dim_to_stack,
                          combine="nested")

# Run the function
s_target_date='01-10-1980'
e_target_date='03-12-2021' # end of year - 28 days
rw_1 = 7
lead_time = 15 # days until valid_time starts
rw = 0 # because the data are not centered
ntimestep = 60 # lags
target_len = len(tp_target)

predictor_array=sel_train_data_lead(pc_xr, target_len, s_target_date, e_target_date,
                rw_1, lead_time, rw, ntimestep)

np_out_name='Predictor_array_crosscor_number.nc'

#np.save(root_results+np_out_name,predictor_array)
predictor_array.to_netcdf(root_results+"/"+np_out_name)

In [ ]:
# hard fix dates intersection
dates = tp_index.time.to_index().intersection(predictor_array.time.to_index())

tp_index = tp_index.sel(time=dates).compute()
tp_target = tp_target.sel(time=dates).compute()
predictor_array = predictor_array.sel(time=dates).compute()

## causal discovery with `tigramite`

In [ ]:
import numpy as np

# install with pip install tigramite
import tigramite
from tigramite import data_processing as pp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, CMIknn, GPDC

# Example data
features_train = predictor_array.sel(lag=0).pcs
sample_size = features_train.time.size
N_features = features_train.mode.size
allX = features_train.values 

# Target
y = tp_target.values

# Construct array needed for tigramite, we need to lag X behind y here just for computation reasons
data = np.hstack((y[:-1].reshape(sample_size-1, 1), allX[1:]))

# Initialize class with ParCorr test, can be changed to nonlinear CI tests, eg CMIknn, but these use more computation time
dataframe = pp.DataFrame(data, var_names = ['target',] + list(range(N_features)))
pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=ParCorr(),  # or CMIknn()  GPDC()
    verbosity=0)

# Set alpha_level for selecting causal features, the smaller the stricter
pc_alpha = 0.01

# Only run on target variable
selected_links = [(i, -1) for i in range(1, N_features + 1)]
causal_predictors = pcmci._run_pc_stable_single(j=0,
                              selected_links=selected_links,
                              tau_min=1,
                              tau_max=1,
                              pc_alpha=pc_alpha)

# Indices of causal features from X
causal_features = [varlag[0] - 1 for varlag in causal_predictors['parents']]
print(causal_features)

In [ ]:
# could subselect these later
predictor_array.isel(mode=causal_features).mode.values

In [ ]:
eofs = xr.open_mfdataset(root_results+"/EOF_*",concat_dim=dim_to_stack,
                          combine="nested")

In [ ]:
#for m in causal_features:
#    eofs.isel(mode=m).eofs.plot(robust=True)
#    plt.show()

## Splitting the data

In [ ]:
drop_OND_years = [2005,2007,2018,2004,2006]
drop_MAM_years = [2009,2001,2002,2005,2020]

def get_train_test_val(data_predictor, data_target, test_frac, val_frac):
    """Splits data across periods into train, test, and validation"""
    # assign the last int(-test_frac*len(tp_predictor)) rows to test data
    test_predictor = data_predictor[int(-test_frac*len(data_target)):]
    test_target = data_target[int(-test_frac*len(data_target)):]
    
    # assign the last int(-test_frac*len(tp_predictor)) from the remaining rows to validation data
    remain_predictor = data_predictor[0:int(-test_frac*len(data_target))]
    remain_target = data_target[0:int(-test_frac*len(data_target))]
    val_predictor = remain_predictor[int(-val_frac*len(remain_predictor)):]
    val_target = remain_target[int(-val_frac*len(remain_predictor)):]
    
    # the remaining rows are assigned to train data
    train_predictor = remain_predictor[:int(-val_frac*len(remain_predictor))]
    train_target = remain_target[:int(-val_frac*len(remain_predictor))]
    return train_predictor, train_target, test_predictor, test_target, val_predictor, val_target

In [ ]:
# define input and output data for LSTM
y_all = keras.utils.to_categorical(tp_index)
X_all = predictor_array.pcs
print(X_all.shape,y_all.shape)

In [ ]:
train_X, train_y, test_X, test_y, val_X, val_y = get_train_test_val(X_all, y_all, test_frac=0.2, val_frac=0.2)

In [ ]:
visualization.plot_split_counts(train_y, val_y, test_y)

## LSTM with attention layer

In [ ]:
# https://github.com/gentaiscool/lstm-attention/blob/58adc7e345b5b3a79638483049704802a66aa1f4/layers.py#L50
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    follows these equations:
    
    (1) u_t = tanh(W h_t + b)
    (2) \alpha_t = \frac{exp(u^T u)}{\sum_t(exp(u_t^T u))}, this is the attention weight
    (3) v_t = \alpha_t * h_t, v in time t
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        3D tensor with shape: `(samples, steps, features)`.
    """

    def __init__(self,
                W_regularizer=None, u_regularizer=None, b_regularizer=None,
                W_constraint=None, u_constraint=None, b_constraint=None,
                bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
                'W_regularizer': self.W_regularizer,
                'u_regularizer': self.u_regularizer,
                'b_regularizer': self.b_regularizer,
                'W_constraint': self.W_constraint,
                'u_constraint': self.u_constraint,
                'b_constraint': self.b_constraint,
                'bias': self.bias,
        })
        return config

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                initializer=self.init,
                                name='{}_W'.format(self.name),
                                regularizer=self.W_regularizer,
                                constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                    initializer='zero',
                                    name='{}_b'.format(self.name),
                                    regularizer=self.b_regularizer,
                                    constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                initializer=self.init,
                                name='{}_u'.format(self.name),
                                regularizer=self.u_regularizer,
                                constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero and this results in NaN's. 
        # Should add a small epsilon as the workaround
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        
        return weighted_input, a

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], input_shape[2]
    
class Addition(Layer):
    """
    This layer is supposed to add of all activation weight.
    We split this from AttentionWithContext to help us getting the activation weights
    follows this equation:
    (1) v = \sum_t(\alpha_t * h_t)
    
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    """

    def __init__(self, **kwargs):
        super(Addition, self).__init__(**kwargs)

    def build(self, input_shape):
        self.output_dim = input_shape[-1]
        super(Addition, self).build(input_shape)

    def call(self, x):
        return K.sum(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
#Create a class weight dictionary to help if the classes are unbalanced
def class_weight_creator(Y):
    class_dict = {}
    weights = np.max(np.sum(Y, axis=0)) / np.sum(Y, axis=0)
    for i in range( Y.shape[-1] ):
        class_dict[i] = weights[i]
        
    return class_dict

In [ ]:
class_weight = class_weight_creator(train_y)

In [ ]:
batch_size = 32
epochs = 5
shuffle = True 
verbose = 2 #Set whether the model will output information when trained (0 = no output; 2 = output accuracy every epoch)

In [ ]:
callbacks_path = '/home/zwu/Lorentz_workshop/test/checkpoint_test'
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath=callbacks_path,
        monitor='val_acc',   # tf.keras.metrics.AUC(from_logits=True)
        save_best_only=True,
    )
]

In [ ]:
# LSTM with attention layer
ntimestep = 60    # number of time step used in the predictors
nfeature = 30   # number of features
input_tensor = Input(shape=(ntimestep,nfeature))
layer1 = layers.LSTM(100, return_sequences=True, kernel_regularizer=regularizers.l2(1))(input_tensor)
layer1 = layers.LSTM(20, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))(layer1)
layer1, alfa = AttentionWithContext()(layer1)
layer1 = Addition()(layer1)
layer1 = layers.Dense(5, activation="relu")(layer1)
output_tensor = layers.Dense(2,activation='softmax')(layer1)

model = Model(input_tensor, output_tensor)
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['acc'])

model.summary()

In [ ]:
# train the model
history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(val_X, val_y), shuffle = shuffle, verbose=verbose, class_weight=class_weight)

In [ ]:
# plot learning curve
train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
fig, (ax1,ax2) = plt.subplots(1,2, figsize=plt.figaspect(0.25))
ax1.plot(train_acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.set_title('Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0,1.1)
ax1.legend()

ax2.plot(train_loss, label='Training loss')
ax2.plot(val_loss, label='Validation loss')
ax2.set_title('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

In [ ]:
# plot learning curve
visualization.plot_learning_curve(history)

In [ ]:
# evaluation
from sklearn.metrics import brier_score_loss
from sklearn.calibration import calibration_curve
test_predict = model.predict(test_X)
y_pred = np.argmax(model.predict(test_X),axis=1)
print('Recall: '+str(round(recall_score(test_y[:,1],y_pred),2)))
print('Precision: '+str(round(precision_score(test_y[:,1],y_pred),2)))
print('F1-score: '+str(round(f1_score(test_y[:,1],y_pred),2)))
print('Accuracy: '+str(round(accuracy_score(test_y[:,1],y_pred),2)))
print('Brier score:' +str(brier_score_loss(test_y[0:-20,1], test_predict[0:-20,1])))

calib_y, calib_x = calibration_curve(test_y[:,1],test_predict[:,1],n_bins=10)
visualization.plot_calibration_curve(calib_x, calib_y)

In [ ]:
visualization.plot_roc_auc(model, test_X, test_y)


In [ ]:
# get weightings of each time step and each sample
intermediate_layer_model2 = Model(inputs=model.input,
                                 outputs=model.layers[3].output)

intermediate_layer_model1 = Model(inputs=model.input,
                                 outputs=model.layers[2].output)

intermediate_layer_model3 = Model(inputs=model.input,
                                 outputs=model.layers[4].output)

intermediate_output2, alfa_output = intermediate_layer_model2.predict(test_X, verbose=0)
intermediate_output1 = intermediate_layer_model1.predict(test_X, verbose=0)
intermediate_output3 = intermediate_layer_model3.predict(test_X, verbose=0)

weights = intermediate_output2 / intermediate_output1
print(np.shape(weights))

In [ ]:
# plot the weights
val_weights = np.ndarray((len(test_X),ntimestep))+np.nan
for ii in range(len(test_X)):
    for j in range(ntimestep):
        val_weights[ii,j] = weights[ii][j][0]
print(np.shape(val_weights))

fig, axs = plt.subplots(1, figsize=plt.figaspect(0.15))
for ii in range(len(test_X)):
    plt.plot(val_weights[ii,:])

fig, axs = plt.subplots(1, figsize=plt.figaspect(0.15))
plt.plot(np.nanmean(val_weights,axis=0),'k')